In [1]:
import matplotlib.pyplot as plt #Plotting
import numpy as np #Number manipulation
import re #String manipulation
import pandas as pd #Dataframe manipulation
import math #Math
import matplotlib #Plotting
#from __future__ import division #Math
import seaborn as sns #Better plotting
import scipy #Statistical tests
from keras.models import Sequential, Input, Model
from keras.layers import LSTM, Dense, Dropout, Flatten, Activation
from keras.layers.normalization import BatchNormalization
from keras.layers.advanced_activations import LeakyReLU
%matplotlib inline 
from ipykernel import kernelapp as app


Using TensorFlow backend.


In [ ]:
# decision making phase all. 

In [2]:
subjects = ["subject12study.csv", "subject13study.csv", "subject14study.csv", "subject15study.csv"]
infos = ["Subject 12.csv", "Subject 13.csv", "Subject 14.csv", "Subject 15.csv"]

data_main = []
data_side = []

for i in range(len(subjects)):
    df = pd.read_csv(subjects[i], delimiter = "\t")
    info = pd.read_csv(infos[i])

    # Cleaning the data
    df = df[df["avgx"] !=0] #eyex and eyey cannot be equal to 0
    df = df[df["avgy"] !=0]
    df = df[df["avgx"] > 0] #eyex should be bigger than 0 and smaller than 1920, regarding the screen size during the study
    df = df[df["avgx"] <1920]
    df = df[df["avgy"] > 0]  
    df = df[df["avgy"] < 1080] #eyey should be bigger than 0 and smaller than 1080, according to the screen size during the study
    df = df[df["psize"] < 25] #pupil size can be max 25 and min 13
    df = df[df["psize"] >13]

    data_main.append(df)
    data_side.append(info)


c:\users\gizem\appdata\local\conda\conda\envs\tensorflow\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (1,2,3) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
datax = []

for q in range(len(data_main)):
    
    realtime = [] #realtime
    for a in data_main[q]["timestamp"]:
        try:
            d , f = a.split()
            (h, m, s) = f.split(':')
            result = float(h) * 3600 + float(m) * 60 + float(s)
            realtime.append(result)
        except:
            realtime.append(float(0))

    len(realtime)
    data_main[q]["realtime"] = realtime
    dfx = data_main[q][["realtime", "avgx", "avgy", "psize"]] #creating a new dataframe for only requiered columns
    
    datax.append(dfx)
    
    imgstart = []
    imgend =[]
    decstart =[]
    decenda = []
    decendb =[]


    for i in data_side[q]["ImgStart"]: #separating the stages of the cross1, context, cross2, images, decision depending on time
            (h,m,s) = i.split(":")
            resulti = float(h)*3600 + float(m)*60 + float(s)
            imgstart.append(resulti)

    for i in data_side[q]["ImgEnd"]: #separating the stages of the cross1, context, cross2, images, decision depending on time
            (h,m,s) = i.split(":")
            resultie = float(h)*3600 + float(m)*60 + float(s)
            imgend.append(resultie)

    for e in data_side[q]["EmptyStart"]: #separating the stages of the cross1, context, cross2, images, decision depending on time
            (h,m,s) = e.split(":")
            resultes = float(h)*3600 + float(m)*60 + float(s)
            decstart.append(resultes)

    for e in data_side[q]["EmptyEnd"]: #separating the stages of the cross1, context, cross2, images, decision depending on time
            (h,m,s) = e.split(":")
            resultee = float(h)*3600 + float(m)*60 + float(s)
            decenda.append(resultee)

    cro2beg = []
    contbeg = []
    cro1beg = []

    for c in data_side[q]["ComplexStart"]: #separating the stages of the cross1, context, cross2, images, decision depending on time
            (h,m,s) = c.split(":")
            result = float(h)*3600 + float(m)*60 + float(s)
            decendb.append(result)
            cro2beg.append(result-9)
            contbeg.append(result-13)
            cro1beg.append(result-14)

    #print (imgstart[1], imgend[1], decstart[1], decenda[1], decendb[1] )

    data_side[q]["Cro1Beg"] = cro1beg  #cross1(fixation)
    data_side[q]["ContBeg"] = contbeg  #context
    data_side[q]["Cro2Beg"] = cro2beg  #cross2(fixation)
    data_side[q]["ImagesBeg"] = imgstart    #images stage started
    data_side[q]["ImagesEnd"] = imgend
    data_side[q]["DecisionBeg"] = decstart   #decision stage started  (green frame around the pictures)
    data_side[q]["DecisionEnd"] = decenda


    data_side[q].sort_values("DecisionEnd", inplace=True, ascending=True) #reorganizing dataframe

    data_side[q] = data_side[q].reset_index(drop=True) #saving df

In [4]:
"""
datac[0][4][79]
for i in range(80):
    print(len(datac[0][4][i]))
"""

'\ndatac[0][4][79]\nfor i in range(80):\n    print(len(datac[0][4][i]))\n'

In [5]:
#len(datax), datax[0].shape, len(data_side), data_side[0].shape
#len(datac), len(datac[0]), len(datac[0][0])
#len(datac), len(datac[0]), len(datac[0][0]), len(datac[0][0][1])

In [6]:

datac = []

for q in range(len(datax)):
    
    cross1 = []
    context = []
    cross2 = []
    images = []
    decision=[]
    sub = []
    
    for i in range(80):
        cross1.append(datax[q].loc[((datax[q]['realtime']) >= float(data_side[q]["Cro1Beg"][i])) & ((datax[q]["realtime"]) <= float(data_side[q]["ContBeg"][i]))])
        context.append(datax[q].loc[((datax[q]['realtime']) >= float(data_side[q]["ContBeg"][i])) & ((datax[q]["realtime"]) <= float(data_side[q]["Cro2Beg"][i]))])
        cross2.append(datax[q].loc[((datax[q]['realtime']) >= float(data_side[q]["Cro2Beg"][i])) & ((datax[q]["realtime"]) <= float(data_side[q]["ImagesBeg"][i]))])
        images.append(datax[q].loc[((datax[q]['realtime']) >= float(data_side[q]["ImagesBeg"][i])) & ((datax[q]["realtime"]) <= float(data_side[q]["ImagesEnd"][i]))])
        decision.append(datax[q].loc[((datax[q]['realtime']) >= float(data_side[q]["DecisionBeg"][i])) & ((datax[q]["realtime"]) <= float(data_side[q]["DecisionEnd"][i]))])
    
    sub.append(cross1)
    sub.append(context)
    sub.append(cross2)
    sub.append(images)
    sub.append(decision)
    
    datac.append(sub)

In [7]:
Cx = [] #choice data as left and right
for i in range(4):
    c = []
    c = data_side[i]['Choice'].tolist()
    Cx.append(c)

In [8]:
len(datac[0][4]), len(Cx[0]) # only decision stage

xdata = []
cdata = []


for i in range(4):
    for q in range(80):
        xdata.append(datac[i][4][q])
        cdata.append(Cx[i][q])
        
print(len(xdata), len(xdata[1]), len(cdata)) # 80*4 80 contexts 4 people


320 115 320


In [9]:
xdata # her context için 100 datapoint 

[       realtime       avgx      avgy     psize
 1941  45324.279  1485.6954  457.4317  16.39720
 1942  45324.312  1491.8469  457.1739  16.52275
 1943  45324.345  1499.1702  457.4417  16.34920
 1944  45324.379  1502.7947  456.3130  16.21080
 1945  45324.411  1505.8673  454.6760  16.19790
 1946  45324.445  1508.0771  453.8122  15.86265
 1947  45324.478  1511.5459  452.5009  15.96270
 1948  45324.511  1514.5326  450.2784  15.62315
 1949  45324.545  1517.5559  450.4314  16.18350
 1950  45324.578  1518.9806  450.7748  15.73755
 1951  45324.612  1518.5045  451.0477  15.94105
 1952  45324.645  1521.5309  450.3058  15.81455
 1953  45324.678  1522.9775  449.1841  15.54570
 1954  45324.711  1524.9199  448.4284  15.41085
 1955  45324.745  1527.1935  448.8989  15.41385
 1956  45324.778  1528.3547  448.0569  15.32005
 1957  45324.812  1529.2959  446.3965  15.31490
 1958  45324.845  1532.9440  445.9150  15.69500
 1959  45324.878  1535.0687  445.6283  15.51355
 1960  45324.911  1536.8302  443.9662  1

In [10]:
nnx = [] #
nnc = []

for i in range(len(xdata)):
    #print(len(xdata[i]), cdata[i])
    
    if len(xdata[i]) >= 80: # her bir subjenin her bir trailının xdata[i] için 
        a = xdata[i][:80] # her bir 320 kutudaki 100 datayı alıyoruz 
        z = a.values[:,1:] # her 1 saniye 20+- datapoints, average 100, cunku decision 4 saniye
        nnx.append(z)
        nnc.append(cdata[i])
    else:
        pass


print(len(nnx), len(nnx[44]), len(nnc)) #total data 249 tane, 320den geriye kalan datalar. 
#print(z.shape, z)



249 80 249


In [11]:
for i in nnx: # Normalize ediyorum datayı
    i[:,0] = i[:,0]/1920
    i[:,1] = i[:,1]/1080
    i[:,2] = (i[:,2]-13)/12

In [12]:
import random 

#1. validation
# modeli train ettiğin datayla değerlendiremezsin çünkü hiç görmediği datayı göstermek datanın ne kadar genellenebilir olduğunu söyler.
#bu yüzden modeldeki validation split i yapıyoruz, yani datamın %20sini modeli eğitmek için kullanma. 
# ama bu yeterli değil validation için, çünkü sen oradaki validation ı yükseltmek için modelin parametreleriyle oynuyorsun.
#2. validation
# ilk validation ı yükseltmek için modelin parametreleriyle oynadığımızdan modeli geliştirmekte hiç kullanmadığımız ayrı bir validation set kullanıyoruz. 
# bu da 30 dataya tekabul ediyor
sayko = list(zip(nnx,nnc)) 
random.shuffle(sayko)
nnx, nnc = zip(*sayko)

valx = nnx[:30]
valy = nnc[:30]

In [13]:
nn_x = np.stack(nnx[30:])
nn_c = np.stack(nnc[30:])
val_x = np.stack(valx)
val_y = np.stack(valy)
print(nn_x.shape, nn_c.shape, val_x.shape, val_y.shape)


(219, 80, 3) (219,) (30, 80, 3) (30,)


In [14]:
f = nn_x.reshape(219,240)
g = val_x.reshape(30,240)
f.shape

(219, 240)

In [15]:
inputs = Input(shape=(240,))

x = Dense(16, activation='relu')(inputs)
x = Dense(8, activation='relu')(x)
prediction = Dense(1, activation='sigmoid')(x)


model = Model(inputs=inputs, outputs=prediction)
model.summary()
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['binary_accuracy'])
history = model.fit(f, nn_c, epochs=100, shuffle= True, batch_size=64, verbose=2, validation_split=0.2 )

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 240)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 16)                3856      
_________________________________________________________________
dense_2 (Dense)              (None, 8)                 136       
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 9         
Total params: 4,001
Trainable params: 4,001
Non-trainable params: 0
_________________________________________________________________
Train on 175 samples, validate on 44 samples
Epoch 1/100
 - 1s - loss: 0.6470 - binary_accuracy: 0.7771 - val_loss: 0.5660 - val_binary_accuracy: 0.8864
Epoch 2/100
 - 0s - loss: 0.5678 - binary_accuracy: 0.8343 - val_loss: 0.5237 - val_binary_accuracy: 0.8636
Epoch 3/100

Epoch 68/100
 - 0s - loss: 0.2275 - binary_accuracy: 0.9200 - val_loss: 0.2077 - val_binary_accuracy: 0.9318
Epoch 69/100
 - 0s - loss: 0.2401 - binary_accuracy: 0.9143 - val_loss: 0.2069 - val_binary_accuracy: 0.9318
Epoch 70/100
 - 0s - loss: 0.2322 - binary_accuracy: 0.9029 - val_loss: 0.2370 - val_binary_accuracy: 0.9318
Epoch 71/100
 - 0s - loss: 0.2288 - binary_accuracy: 0.9200 - val_loss: 0.2180 - val_binary_accuracy: 0.9091
Epoch 72/100
 - 0s - loss: 0.2262 - binary_accuracy: 0.9200 - val_loss: 0.2186 - val_binary_accuracy: 0.9091
Epoch 73/100
 - 0s - loss: 0.2267 - binary_accuracy: 0.9143 - val_loss: 0.2117 - val_binary_accuracy: 0.9091
Epoch 74/100
 - 0s - loss: 0.2304 - binary_accuracy: 0.9086 - val_loss: 0.2573 - val_binary_accuracy: 0.9318
Epoch 75/100
 - 0s - loss: 0.2271 - binary_accuracy: 0.9314 - val_loss: 0.2059 - val_binary_accuracy: 0.9318
Epoch 76/100
 - 0s - loss: 0.2254 - binary_accuracy: 0.9257 - val_loss: 0.2122 - val_binary_accuracy: 0.9091
Epoch 77/100
 - 0s 

In [16]:
inputs = Input(shape=(240,))

x = Dense(8, activation='relu')(inputs)
x = Dense(4, activation='relu')(x)
prediction = Dense(1, activation='sigmoid')(x)


model2 = Model(inputs=inputs, outputs=prediction)
model2.summary()
model2.compile(optimizer='adam', loss='mse', metrics=['binary_accuracy'])
history = model2.fit(f, nn_c, epochs=100, shuffle= True, batch_size=64, verbose=2, validation_split=0.2 )

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 240)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 8)                 1928      
_________________________________________________________________
dense_5 (Dense)              (None, 4)                 36        
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 5         
Total params: 1,969
Trainable params: 1,969
Non-trainable params: 0
_________________________________________________________________
Train on 175 samples, validate on 44 samples
Epoch 1/100
 - 1s - loss: 0.2501 - binary_accuracy: 0.5086 - val_loss: 0.2246 - val_binary_accuracy: 0.5227
Epoch 2/100
 - 0s - loss: 0.2312 - binary_accuracy: 0.5086 - val_loss: 0.2205 - val_binary_accuracy: 0.5227
Epoch 3/100

Epoch 68/100
 - 0s - loss: 0.1250 - binary_accuracy: 0.9143 - val_loss: 0.1055 - val_binary_accuracy: 0.9545
Epoch 69/100
 - 0s - loss: 0.1245 - binary_accuracy: 0.9086 - val_loss: 0.1041 - val_binary_accuracy: 0.9545
Epoch 70/100
 - 0s - loss: 0.1236 - binary_accuracy: 0.9086 - val_loss: 0.1036 - val_binary_accuracy: 0.9545
Epoch 71/100
 - 0s - loss: 0.1230 - binary_accuracy: 0.9143 - val_loss: 0.1040 - val_binary_accuracy: 0.9545
Epoch 72/100
 - 0s - loss: 0.1221 - binary_accuracy: 0.9257 - val_loss: 0.1036 - val_binary_accuracy: 0.9545
Epoch 73/100
 - 0s - loss: 0.1214 - binary_accuracy: 0.9257 - val_loss: 0.1027 - val_binary_accuracy: 0.9545
Epoch 74/100
 - 0s - loss: 0.1207 - binary_accuracy: 0.9257 - val_loss: 0.1008 - val_binary_accuracy: 0.9545
Epoch 75/100
 - 0s - loss: 0.1200 - binary_accuracy: 0.9200 - val_loss: 0.1002 - val_binary_accuracy: 0.9545
Epoch 76/100
 - 0s - loss: 0.1195 - binary_accuracy: 0.9257 - val_loss: 0.1004 - val_binary_accuracy: 0.9545
Epoch 77/100
 - 0s 

In [17]:
s = model.predict(g)

In [18]:
model.evaluate(x=g, y=val_y, verbose=1, sample_weight=None, steps=1000)

1000/1000 [==============================] - 2s 2ms/step


[0.4288117587566376, 0.800000011920929]

In [19]:
model2.evaluate(x=g, y=val_y, verbose=1, sample_weight=None, steps=1000)

1000/1000 [==============================] - 1s 802us/step


[0.15177904069423676, 0.8333333134651184]

In [20]:
for i in range(len(val_y)):
    if s[i] > 0.5:
        print('Model Predicted: ', 1, ' Real Decision: ', val_y[i])
    else:
        print('Model Predicted: ', 0, ' Real Decision: ', val_y[i])

Model Predicted:  0  Real Decision:  0.0
Model Predicted:  0  Real Decision:  1.0
Model Predicted:  1  Real Decision:  1.0
Model Predicted:  0  Real Decision:  1.0
Model Predicted:  1  Real Decision:  1.0
Model Predicted:  1  Real Decision:  1.0
Model Predicted:  1  Real Decision:  1.0
Model Predicted:  1  Real Decision:  1.0
Model Predicted:  0  Real Decision:  0.0
Model Predicted:  1  Real Decision:  1.0
Model Predicted:  0  Real Decision:  0.0
Model Predicted:  1  Real Decision:  1.0
Model Predicted:  1  Real Decision:  1.0
Model Predicted:  1  Real Decision:  1.0
Model Predicted:  1  Real Decision:  1.0
Model Predicted:  1  Real Decision:  1.0
Model Predicted:  0  Real Decision:  0.0
Model Predicted:  1  Real Decision:  1.0
Model Predicted:  0  Real Decision:  0.0
Model Predicted:  0  Real Decision:  0.0
Model Predicted:  1  Real Decision:  0.0
Model Predicted:  0  Real Decision:  0.0
Model Predicted:  1  Real Decision:  1.0
Model Predicted:  1  Real Decision:  1.0
Model Predicted: